# Heart Classification End-to-End

This notebook demonstrates a complete ML lifecycle:
1.  **Database Normalization**: Loading CSV data into a normalized SQLite database.
2.  **Data Loading**: Reconstructing the dataset via SQL.
3.  **EDA & Preprocessing**: Analyzing class balance and building a pipeline.
4.  **Experiment Tracking**: Running 16 experiments (Algorithm x PCA x Tuning) with MLflow.
5.  **Model Selection**: Saving the best model based on F1 score.


## 1. Setup and Imports


In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
import joblib
import json
import mlflow
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.base import clone

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Define paths
DATA_DIR = '../Data'
DB_PATH = os.path.join(DATA_DIR, 'heart.db')
CSV_PATH = os.path.join(DATA_DIR, 'heart.csv')
MODELS_DIR = '../models'

os.makedirs(MODELS_DIR, exist_ok=True)


c:\Users\emman\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Database Creation & Normalization

We will read the raw CSV and convert it into a normalized SQLite schema.

**Schema Design**:
-   `patients`: `id` (generated), `age`, `sex`
-   `lookup_cp`, `lookup_restecg`, `lookup_slope`, `lookup_thal`: Reference tables for categorical codes.
-   `exams`: Foreign keys to patients and lookups, plus remaining measurements.


In [2]:
def create_normalized_db(csv_path, db_path):
    if os.path.exists(db_path):
        os.remove(db_path)
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 1. Read Raw Data
    df = pd.read_csv(csv_path)
    
    # Generate Patient IDs (assuming one row per patient for this dataset)
    df['patient_id'] = range(1, len(df) + 1)
    
    # 2. Extract Lookup Tables
    # CP
    cp_values = sorted(df['cp'].unique())
    cursor.execute("CREATE TABLE lookup_cp (cp_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_cp (cp_code, description) VALUES (?, ?)", 
                       [(int(x), f"cp_{x}") for x in cp_values])
                       
    # RestECG
    restecg_values = sorted(df['restecg'].unique())
    cursor.execute("CREATE TABLE lookup_restecg (restecg_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_restecg (restecg_code, description) VALUES (?, ?)", 
                       [(int(x), f"restecg_{x}") for x in restecg_values])
                       
    # Slope
    slope_values = sorted(df['slope'].unique())
    cursor.execute("CREATE TABLE lookup_slope (slope_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_slope (slope_code, description) VALUES (?, ?)", 
                       [(int(x), f"slope_{x}") for x in slope_values])
                       
    # Thal
    thal_values = sorted(df['thal'].unique())
    cursor.execute("CREATE TABLE lookup_thal (thal_code INTEGER PRIMARY KEY, description TEXT)")
    cursor.executemany("INSERT INTO lookup_thal (thal_code, description) VALUES (?, ?)", 
                       [(int(x), f"thal_{x}") for x in thal_values])

    # 3. Create Patients Table
    # Storing Age and Sex separately as requested
    cursor.execute('''
        CREATE TABLE patients (
            patient_id INTEGER PRIMARY KEY,
            age INTEGER,
            sex INTEGER
        )
    ''')
    patients_data = df[['patient_id', 'age', 'sex']].drop_duplicates()
    patients_data.to_sql('patients', conn, if_exists='append', index=False)
    
    # 4. Create Exams Table (The Main Table)
    # References patients and lookups
    cursor.execute('''
        CREATE TABLE exams (
            exam_id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id INTEGER,
            cp INTEGER,
            trestbps INTEGER,
            chol INTEGER,
            fbs INTEGER,
            restecg INTEGER,
            thalach INTEGER,
            exang INTEGER,
            oldpeak REAL,
            slope INTEGER,
            ca INTEGER,
            thal INTEGER,
            target INTEGER,
            FOREIGN KEY(patient_id) REFERENCES patients(patient_id),
            FOREIGN KEY(cp) REFERENCES lookup_cp(cp_code),
            FOREIGN KEY(restecg) REFERENCES lookup_restecg(restecg_code),
            FOREIGN KEY(slope) REFERENCES lookup_slope(slope_code),
            FOREIGN KEY(thal) REFERENCES lookup_thal(thal_code)
        )
    ''')
    
    # Prepare exams data
    exams_cols = ['patient_id', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 
                  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']
    exams_data = df[exams_cols]
    exams_data.to_sql('exams', conn, if_exists='append', index=False)
    
    conn.commit()
    conn.close()
    print(f"Database created at {db_path}")

# Run the creation function
create_normalized_db(CSV_PATH, DB_PATH)


Database created at ../Data\heart.db


## 3. Data Loading (Reconstruction)

We reconstruct the training DataFrame by joining the normalized tables.


In [3]:
def load_data_from_db(db_path):
    conn = sqlite3.connect(db_path)
    
    query = '''
        SELECT 
            p.age, p.sex,
            e.cp, e.trestbps, e.chol, e.fbs, e.restecg, 
            e.thalach, e.exang, e.oldpeak, e.slope, e.ca, e.thal, 
            e.target
        FROM exams e
        JOIN patients p ON e.patient_id = p.patient_id
    '''
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

item_df = load_data_from_db(DB_PATH)
print("Data shape:", item_df.shape)
item_df.head()


Data shape: (1025, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


## 4. EDA & Preprocessing

Basic checks and pipeline construction.


In [5]:
# Class Balance
print("Class Balance:")
print(item_df['target'].value_counts(normalize=True))

# Summary Stats
print("Summary Stats:")
print(item_df.describe())


Class Balance:
target
1    0.513171
0    0.486829
Name: proportion, dtype: float64
Summary Stats:
               age          sex           cp     trestbps        chol  \
count  1025.000000  1025.000000  1025.000000  1025.000000  1025.00000   
mean     54.434146     0.695610     0.942439   131.611707   246.00000   
std       9.072290     0.460373     1.029641    17.516718    51.59251   
min      29.000000     0.000000     0.000000    94.000000   126.00000   
25%      48.000000     0.000000     0.000000   120.000000   211.00000   
50%      56.000000     1.000000     1.000000   130.000000   240.00000   
75%      61.000000     1.000000     2.000000   140.000000   275.00000   
max      77.000000     1.000000     3.000000   200.000000   564.00000   

               fbs      restecg      thalach        exang      oldpeak  \
count  1025.000000  1025.000000  1025.000000  1025.000000  1025.000000   
mean      0.149268     0.529756   149.114146     0.336585     1.071512   
std       0.356527    

In [6]:
# Define Preprocessing Pipeline

X = item_df.drop('target', axis=1)
y = item_df['target']

# Categorical columns to encode
cat_features = ['cp', 'restecg', 'slope', 'thal']
# Numeric columns to scale (all others except sex, fbs, exang which are binary but scaling them is fine too usually, or pass through)
# For simplicity, we'll scale all non-categorical features.
num_features = [c for c in X.columns if c not in cat_features]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=RANDOM_SEED)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (820, 13)
Test shape: (205, 13)


## 5. Experiment Loop (16 Runs)

Algorithms: LogReg, RF, SVC, GBM.
Conditions: +/- PCA, +/- Optuna.


In [9]:
pip install -q dagshub mlflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import dagshub
dagshub.init(repo_owner="emmu9520", repo_name="my-first-repo", mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=06d8c4a8-e805-4987-bd68-05a617d61c65&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=704baafdf7333cc3a2d8369664cf7e96126ab20d2aea90ea2f5375a3fd393e16




Accessing as emmu9520

Initialized MLflow to track repo "emmu9520/my-first-repo"

Repository emmu9520/my-first-repo initialized!

In [12]:
# Define Base Models
models_registry = {
    'LogisticRegression': LogisticRegression(random_state=RANDOM_SEED, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=RANDOM_SEED),
    'SVC': SVC(random_state=RANDOM_SEED, probability=True),
    'GradientBoosting': GradientBoostingClassifier(random_state=RANDOM_SEED)
}

# Optuna Objective Functions
def objective(trial, algo_name, X, y):
    if algo_name == 'LogisticRegression':
        C = trial.suggest_loguniform('C', 1e-4, 1e2)
        model = LogisticRegression(C=C, random_state=RANDOM_SEED, max_iter=1000)
    elif algo_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        max_depth = trial.suggest_int('max_depth', 2, 20)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=RANDOM_SEED)
    elif algo_name == 'SVC':
        C = trial.suggest_loguniform('C', 1e-3, 1e2)
        gamma = trial.suggest_loguniform('gamma', 1e-3, 1e2)
        model = SVC(C=C, gamma=gamma, random_state=RANDOM_SEED, probability=True)
    elif algo_name == 'GradientBoosting':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
        max_depth = trial.suggest_int('max_depth', 2, 10)
        model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=RANDOM_SEED)
    else:
        return 0
        
    score = cross_val_score(model, X, y, cv=3, scoring='f1').mean()
    return score

# Helper to capture Optuna logs to prevent clutter
optuna.logging.set_verbosity(optuna.logging.WARNING)

experiment_results = []
mlflow_experiment_name = "heart_classification_16_runs"
mlflow.set_experiment(mlflow_experiment_name)

# Ensure MLflow tracking URI is set (can be done via env vars outside notebook, or explicitly here if passed)
# os.environ["MLFLOW_TRACKING_URI"] = "..." 

# THE LOOP
algorithms = ['LogisticRegression', 'RandomForest', 'SVC', 'GradientBoosting']
pca_flags = [False, True]
tuning_flags = [False, True]

run_count = 0

for algo in algorithms:
    for use_pca in pca_flags:
        for use_tuning in tuning_flags:
            run_count += 1
            run_name = f"{algo}_{'PCA' if use_pca else 'NoPCA'}_{'Tuned' if use_tuning else 'Default'}"
            print(f"Running scenario {run_count}/16: {run_name}")
            
            with mlflow.start_run(run_name=run_name):
                # 1. Build Preprocessing Step of Pipeline
                steps = [('preprocessor', preprocessor)]
                
                if use_pca:
                    # We pick n_components=0.95 (variance explained) as a sensible pivot for PCA
                    steps.append(('pca', PCA(n_components=0.95, random_state=RANDOM_SEED)))
                
                # Combine preprocessing so we can feed it to Optuna if needed (Optuna needs to CV on transformed data or use pipeline)
                # To keep it simple + correct: We used a Pipeline for the final model. 
                # For Optuna optimization, we'll optimize the CLASSIFIER params.
                # We should pre-transform X_train to speed up Optuna, or put the whole thing in a pipeline inside CV.
                # Let's use pre-transformed X for Optuna to save time.
                
                prep_pipe = Pipeline(steps)
                X_train_transformed = prep_pipe.fit_transform(X_train, y_train)
                
                # 2. Determine Classifier & Params
                best_params = {}
                clf = None
                
                if use_tuning:
                    study = optuna.create_study(direction='maximize')
                    study.optimize(lambda trial: objective(trial, algo, X_train_transformed, y_train), n_trials=10) # 10 trials for speed
                    best_params = study.best_params
                    # Instantiate model with best params
                    if algo == 'LogisticRegression':
                        clf = LogisticRegression(**best_params, random_state=RANDOM_SEED, max_iter=1000)
                    elif algo == 'RandomForest':
                        clf = RandomForestClassifier(**best_params, random_state=RANDOM_SEED)
                    elif algo == 'SVC':
                        clf = SVC(**best_params, random_state=RANDOM_SEED, probability=True)
                    elif algo == 'GradientBoosting':
                        clf = GradientBoostingClassifier(**best_params, random_state=RANDOM_SEED)
                else:
                    clf = clone(models_registry[algo])
                    best_params = "default"

                # 3. Create Final Pipeline & Fit
                # We need to rebuild the full pipeline (Prep [+ PCA] + Classifier)
                # We reuse the `steps` list which has prep and maybe pca
                full_pipeline = Pipeline(steps + [('classifier', clf)])
                
                full_pipeline.fit(X_train, y_train)
                
                # 4. Evaluate
                y_pred = full_pipeline.predict(X_test)
                f1 = f1_score(y_test, y_pred)
                acc = accuracy_score(y_test, y_pred)
                
                print(f"  --> F1: {f1:.4f}")
                
                # 5. Log to MLflow
                mlflow.log_param("algorithm", algo)
                mlflow.log_param("use_pca", use_pca)
                mlflow.log_param("use_tuning", use_tuning)
                mlflow.log_params(best_params if isinstance(best_params, dict) else {"params": "default"})
                
                mlflow.log_metric("f1_score", f1)
                mlflow.log_metric("accuracy", acc)
                
                # Save Artifacts locally first then log
                # Model
                model_filename = f"model_{run_name}.joblib"
                joblib.dump(full_pipeline, model_filename)
                mlflow.log_artifact(model_filename)
                
                # Metrics JSON
                metrics = {"f1_score": f1, "accuracy": acc}
                metrics_filename = f"metrics_{run_name}.json"
                with open(metrics_filename, "w") as f:
                    json.dump(metrics, f)
                mlflow.log_artifact(metrics_filename)
                
                # Metadata JSON
                meta = {
                    "run_name": run_name,
                    "algorithm": algo,
                    "use_pca": use_pca,
                    "use_tuning": use_tuning,
                    "best_params": best_params
                }
                meta_filename = f"metadata_{run_name}.json"
                with open(meta_filename, "w") as f:
                    json.dump(meta, f)
                mlflow.log_artifact(meta_filename)
                
                # Cleanup local files
                os.remove(model_filename)
                os.remove(metrics_filename)
                os.remove(meta_filename)
                
                # Store in results list for Final Selection
                experiment_results.append({
                    "run_name": run_name,
                    "f1": f1,
                    "model": full_pipeline,
                    "metadata": meta
                })

print("All experiments completed.")


2025/12/18 21:29:27 INFO mlflow.tracking.fluent: Experiment with name 'heart_classification_16_runs' does not exist. Creating a new experiment.


Running scenario 1/16: LogisticRegression_NoPCA_Default
  --> F1: 0.8700
🏃 View run LogisticRegression_NoPCA_Default at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/ec30c199745948d5b298535b332f31a7
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 2/16: LogisticRegression_NoPCA_Tuned


C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipyker

  --> F1: 0.8700
🏃 View run LogisticRegression_NoPCA_Tuned at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/00ea46760fbd459690613d900d842755
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 3/16: LogisticRegression_PCA_Default
  --> F1: 0.8279
🏃 View run LogisticRegression_PCA_Default at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/369f949208384075955f5bd47771a26a
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 4/16: LogisticRegression_PCA_Tuned


C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-4, 1e2)
C:\Users\emman\AppData\Local\Temp\ipyker

...

  --> F1: 1.0000
🏃 View run SVC_PCA_Tuned at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/26b418ef43384bb78a5470d5b5ee1da6
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 13/16: GradientBoosting_NoPCA_Default
  --> F1: 0.9714
🏃 View run GradientBoosting_NoPCA_Default at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/16a2831063c4404d9c33c2a350e2af2f
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 14/16: GradientBoosting_NoPCA_Tuned


C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logun

  --> F1: 1.0000
🏃 View run GradientBoosting_NoPCA_Tuned at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/6ee969816008440dba7155a327d18a23
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 15/16: GradientBoosting_PCA_Default
  --> F1: 0.9906
🏃 View run GradientBoosting_PCA_Default at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/c0c54b9768bd4ef9bbd990736700280c
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
Running scenario 16/16: GradientBoosting_PCA_Tuned


C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.001, 0.1)
C:\Users\emman\AppData\Local\Temp\ipykernel_21924\1528894872.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logun

  --> F1: 1.0000
🏃 View run GradientBoosting_PCA_Tuned at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0/runs/96c9c058af9548158eba3c57e287968e
🧪 View experiment at: https://dagshub.com/emmu9520/my-first-repo.mlflow/#/experiments/0
All experiments completed.


## 6. Best Model Selection

Identifying the run with the highest F1 score and saving it.


In [13]:
# Find best run
best_run = max(experiment_results, key=lambda x: x['f1'])

print(f"Best Run: {best_run['run_name']} with F1: {best_run['f1']:.4f}")

# Save Best Model
best_model_path = os.path.join(MODELS_DIR, "best_model.joblib")
joblib.dump(best_run['model'], best_model_path)

# Save Best Metadata
best_meta_path = os.path.join(MODELS_DIR, "best_model_metadata.json")
with open(best_meta_path, "w") as f:
    json.dump(best_run['metadata'], f)

print(f"Saved best model to {best_model_path}")
print(f"Saved best metadata to {best_meta_path}")


Best Run: RandomForest_NoPCA_Default with F1: 1.0000
Saved best model to ../models\best_model.joblib
Saved best metadata to ../models\best_model_metadata.json
